In [2]:
import pymongo
import csv
import charts

from datetime import datetime


Server running in the folder d:\Backup\leiyue\Desktop\实战计划\Plan-for-combating\week3大作业提交\leiyue at 127.0.0.1:2016


In [3]:
client = pymongo.MongoClient('localhost', 27017)
scrapy = client['scrapy']
item_info = scrapy['item_infoY']


In [83]:
start_date = datetime(2000,1,10)
end_date = datetime(2016,1,16)
for i in item_info.find(dict(pub_date={'$gte': start_date, '$lte': end_date}),dict(cates={'$slice': [2, 1]})).sort('pub_date', pymongo.DESCENDING).limit(300):
    print(i)

{'url': 'http://bj.58.com/ershoushebei/21121737272709x.shtml', '_id': ObjectId('569981c2a98063de500b967b'), 'cates': ['北京二手设备'], 'title': '【图】机床打腻子、喷漆、翻新 - 北京周边二手设备 - 北京58同城', 'area': ['北京周边'], 'look': '9成新', 'price': '面议', 'pub_date': datetime.datetime(2016, 1, 16, 0, 0)}
{'url': 'http://bj.58.com/wenti/24620367860422x.shtml', '_id': ObjectId('569981cca98063de4b0b968a'), 'cates': ['北京二手文体/户外/乐器'], 'title': '【图】户外求生刀 黑曼巴 - 房山阎村文体/户外/乐器 - 北京58同城', 'area': ['房山', '阎村'], 'look': '-', 'price': '300 元', 'pub_date': datetime.datetime(2016, 1, 16, 0, 0)}
{'url': 'http://bj.58.com/tiaozao/24353589250484x.shtml', '_id': ObjectId('56998209a98063de4d0b96bb'), 'cates': ['北京其他二手物品'], 'title': '【图】男士霸气龙头链子 一秒变男神 - 朝阳其他二手物品 - 北京58同城', 'area': ['朝阳'], 'look': '全新', 'price': '500 元', 'pub_date': datetime.datetime(2016, 1, 16, 0, 0)}
{'url': 'http://bj.58.com/ershoujiaju/24560953563444x.shtml', '_id': ObjectId('5699820ca98063de4e0b96c6'), 'cates': ['北京二手家具'], 'title': '【图】铁艺落地衣帽架，50元转让 - 丰台蒲黄榆二手家具 - 北京5

In [109]:
pipeline = [
    {'$match': {'$and': [dict(pub_date={'$gte': start_date, '$lte': end_date}), dict(area='海淀')]}},
    {'$project': dict(cate={'$slice': [ '$cates', 2, 1]})}, 
    {'$group': dict(_id='$cate', counts={'$sum': 1})},
    {'$unwind': '$_id' },
    {'$sort': dict(counts=pymongo.DESCENDING)},
    {'$limit': 99}
]
# print(pipeline)

In [89]:
#for i in item_info.find():
#    date = datetime.strptime(i['pub_date'], '%Y-%m-%d')
#    item_info.update_one(dict(_id=i['_id']), {'$set': dict(pub_date=date)})

In [110]:
for i in item_info.aggregate(pipeline):
    # print(i['counts'], i['_id'])
    data = {
            'name': i['_id'],
            'data': [ i['counts'] ],
            'type': 'column'
        }
    print(data)

{'data': [2452], 'type': 'column', 'name': '北京二手家电'}
{'data': [1690], 'type': 'column', 'name': '北京二手数码产品'}
{'data': [1206], 'type': 'column', 'name': '北京二手服装/鞋帽/箱包'}
{'data': [1123], 'type': 'column', 'name': '北京二手文体/户外/乐器'}
{'data': [1011], 'type': 'column', 'name': '北京二手办公用品/设备'}
{'data': [969], 'type': 'column', 'name': '北京二手母婴/儿童用品'}
{'data': [700], 'type': 'column', 'name': '北京二手家具'}
{'data': [663], 'type': 'column', 'name': '北京二手台式机/配件'}
{'data': [477], 'type': 'column', 'name': '北京二手笔记本'}
{'data': [410], 'type': 'column', 'name': '北京二手图书/音像/软件'}
{'data': [268], 'type': 'column', 'name': '北京二手手机'}
{'data': [255], 'type': 'column', 'name': '北京二手平板电脑'}
{'data': [252], 'type': 'column', 'name': '北京二手美容/保健'}
{'data': [189], 'type': 'column', 'name': '北京其他二手物品'}
{'data': [103], 'type': 'column', 'name': '北京二手设备'}


In [4]:
def generator(start_date, end_date, area):
    pipeline = [
        {'$match': {'$and': [dict(pub_date={'$gte': start_date, '$lte': end_date}), dict(area=area)]}},
        {'$project': dict(cate={'$slice': [ '$cates', 2, 1]})}, 
        {'$group': dict(_id='$cate', counts={'$sum': 1})},
        {'$unwind': '$_id' },
        {'$sort': dict(counts=pymongo.DESCENDING)},
        {'$limit': 99}
    ]
    for i in item_info.aggregate(pipeline):
        item = {
            'name': i['_id'],
            'data': [ i['counts'] ],
            'type': 'column',
            'area': area
        }
        yield item

In [113]:
start_date = datetime(2000,1,10)
end_date = datetime(2016,1,16)

options = dict(
    title=dict(text='发帖量类目统计'),
    chart=dict(zoomType='xy'),
    xAxis=dict(title=dict(text='日期')),
    yAxis=dict(title=dict(text='数量'))
)

series = [ i for i in generator(start_date, end_date, '海淀') ]

charts.plot(series, options=options, show='inline')

In [6]:
start_date = datetime(2000,1,10)
end_date = datetime(2016,1,16)
area= '朝阳'

headers = ['area', 'name', 'data', 'type']

options = dict(
    title=dict(text=area + '发帖量类目统计'),
    chart=dict(zoomType='xy'),
    xAxis=dict(title=dict(text='日期')),
    yAxis=dict(title=dict(text='数量'))
)

series = [ i for i in generator(start_date, end_date, area) ]

   
with open(area+'.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(series)

charts.plot(series, options=options, show='inline')

